In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.layers import Bidirectional
from sklearn.metrics import r2_score
from tensorflow import keras

Load dataframes

In [2]:
df1 = pd.read_csv (r'C:\Users\Manos\OneDrive/Thesis Final/LSTM/Scaled_and_unidemensiolized_datasets\wave2.tab',sep='\t',low_memory=False)
df2 = pd.read_csv (r'C:\Users\Manos\OneDrive/Thesis Final/LSTM/Scaled_and_unidemensiolized_datasets\wave4.tab',sep='\t',low_memory=False)
df3 = pd.read_csv (r'C:\Users\Manos\OneDrive/Thesis Final/LSTM/Scaled_and_unidemensiolized_datasets\wave6.tab',sep='\t',low_memory=False)
df4 = pd.read_csv (r'C:\Users\Manos\OneDrive/Thesis Final/LSTM/Scaled_and_unidemensiolized_datasets\wave8.tab',sep='\t',low_memory=False)

In [3]:
Dict= {1:df1,
       2:df2,
       3:df3,
       4:df4
       }

<h4> Prapare our 3d array for input</h4>
I use the equal shaped datasets that i have prepared earlier
for every sample's features seiries convert I append them to an array per wave

In [4]:
masterlist=[]
for i in range (0,Dict[1].shape[0]):
    featurelist=[]
    for k in range (1,len(Dict)+1):
        featurelist.append(Dict[k].iloc[i].squeeze())
    featurearr=np.array(featurelist) # convert list to np.array
    masterlist.append(featurearr)#add array of seiries to masterlist
Dataset=np.array(masterlist)

In [5]:
Dataset.shape

(2820, 4, 25)

In [6]:
n_steps=3
X=Dataset[:,:n_steps,1:] #all individuals except id col and last step
y=Dataset[:,n_steps,1:] #last dataset

In [7]:
print('X shape == {}.'.format(X.shape))
print('y shape == {}.'.format(y.shape))

X shape == (2820, 3, 24).
y shape == (2820, 24).


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=32,train_size=0.8)

opt = SGD(learning_rate=0.01)<h2> Define Model </h2>

In [9]:
# opt = SGD(learning_rate=0.001)
opt = keras.optimizers.Adam(learning_rate=0.00005)

In [10]:
# n_epochs=50
# lr= 0.001
# decay_rate = lr / n_epochs
# momentum = 0.8
# opt = SGD(learning_rate=lr, momentum=momentum, decay=decay_rate, nesterov=False)

In [11]:
# opt = keras.optimizers.Adam(learning_rate=0.00005)
# model= Sequential()
# model.add(Conv1D(filters=24, kernel_size=2, activation='relu', input_shape=(X_train.shape[1],X_train.shape[2])))
# model.add(Conv1D(filters=24, kernel_size=1, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(RepeatVector(1))
# model.add(LSTM(24,return_sequences=True,activation='relu'))
# model.add(LSTM (12,activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(X.shape[2], activation='linear'))
# model.compile(loss = "mse", optimizer = opt , metrics=['mae'])




In [12]:

opt = keras.optimizers.Adam(learning_rate=0.00005)
model= Sequential()
model.add(Conv1D(filters=24, kernel_size=2, activation='relu', input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(RepeatVector(1))
model.add(LSTM(24,return_sequences=True,activation='relu'))
model.add(LSTM (12,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(X.shape[2], activation='linear'))
model.compile(loss = "mse", optimizer = opt , metrics=['mae'])


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2, 24)             1176      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2, 24)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 48)                0         
                                                                 
 repeat_vector (RepeatVector  (None, 1, 48)            0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 1, 24)             7008      
                                                                 
 lstm_1 (LSTM)               (None, 12)                1

<h4> fit the model </h4>

In [14]:
history = model.fit(X_train, y_train, epochs=35,validation_split=0.2, batch_size=8, verbose=1)

Epoch 1/35
226/226 [==============================] - 3s 3ms/step - loss: 0.0697 - mae: 0.1864 - val_loss: 0.0703 - val_mae: 0.1825
Epoch 2/35
226/226 [==============================] - 0s 2ms/step - loss: 0.0648 - mae: 0.1746 - val_loss: 0.0647 - val_mae: 0.1700
Epoch 3/35
226/226 [==============================] - 1s 2ms/step - loss: 0.0578 - mae: 0.1619 - val_loss: 0.0550 - val_mae: 0.1562
Epoch 4/35
226/226 [==============================] - 1s 2ms/step - loss: 0.0457 - mae: 0.1478 - val_loss: 0.0413 - val_mae: 0.1371
Epoch 5/35
226/226 [==============================] - 0s 2ms/step - loss: 0.0354 - mae: 0.1292 - val_loss: 0.0326 - val_mae: 0.1161
Epoch 6/35
226/226 [==============================] - 0s 2ms/step - loss: 0.0284 - mae: 0.1136 - val_loss: 0.0266 - val_mae: 0.1005
Epoch 7/35
226/226 [==============================] - 0s 2ms/step - loss: 0.0239 - mae: 0.1015 - val_loss: 0.0223 - val_mae: 0.0883
Epoch 8/35
226/226 [==============================] - 0s 2ms/step - loss: 0.

KeyboardInterrupt: 

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs= range(1,len(loss)+1)
plt.plot(epochs, loss, 'y',label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('Loss_for_wave8_output.png')
plt.show

In [ ]:
acc= history.history['mae']
val_acc= history.history['val_mae']
plt.plot(epochs,acc, 'b', label='Trainining MAE')
plt.plot(epochs, val_acc, 'g', label='Validation MAE')
plt.title('Training and validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.savefig(r'Accuracy_for_wave8_output.png')
plt.show()

In [ ]:
acc= history.history['mae']
val_acc= history.history['val_mae']
plt.plot(epochs,acc, 'b', label='Trainining MAE')
plt.plot(epochs, val_acc, 'g', label='Validation MAE')
plt.title('Training and validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.savefig(r'Accuracy_for_wave8_output.png')
plt.show()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error,mean_squared_error
mse=mean_squared_error(y_pred, y_test)
mae = mean_absolute_error(y_pred, y_test)
mape = mean_absolute_percentage_error(y_pred, y_test)
r2=r2_score(y_pred, y_test)
adjr2=1-(((1-r2)*(len(y)-1))/((len(y)-X.shape[1]-1)))
print('MSE: %.3f' % mse)
print('MAE: %.3f' % mae)
print('MAPE: %.3f' % mape)
print('r2: %.3f' % r2)
print('adjr2: %.3f' % adjr2)

LSTM + conv +adam

LSTM + conv opt
MSE: 0.006
MAE: 0.039
MAPE: 0.416
r2: -64.363
adjr2: -64.432


LSTM opt
MSE: 0.005
MAE: 0.036
MAPE: 0.303
r2: -132.508
adjr2: -132.650

In [ ]:
#load scaler back
from pickle import load
scaler = load(open('scaler.pkl', 'rb'))

In [ ]:
 #reverse transform problem with underfitting
#pred=scaler.inverse_transform(pred)


make array to pd frame

In [ ]:
feat_list=['idauniq','sys1','sys2','sys3', 'dias1','dias2', 'dias3','pulse1','pulse2','pulse3','cfib','chol','hdl','trig','ldl',
           'rtin','hscrp','hgb','hba1c','mmgsd1','mmgsd2','mmgsd3','mmgsn1','mmgsn2','mmgsn3']
#pred = pd.DataFrame(pred, columns = feat_list[1:])

merge IDs again

In [ ]:
pred = pd.concat([df2.iloc[:,0], pred], axis=1, join="inner")

unscale original dataset

In [ ]:
#df4=scaler.inverse_transform(df4.iloc[:,1:])

make df4 as pd

In [ ]:
#df4 = pd.DataFrame(df4, columns = feat_list[1:])
#df4 = pd.concat([df2.iloc[:,0], df4], axis=1, join="inner")

In [ ]:
r2_score(df3.iloc[:,1:], pred)

In [ ]:
abjr2=1-(1-0.19)

In [ ]:
#pred.to_csv(r'C:\Users\Manos/OneDrive/Thesis Final/LSTM/Scaled_and_unidemensiolized_datasets\predicted_wave8.tab',sep='\t',index=False)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(df4, pred)

In [ ]:
pred

In [ ]:
df4.iloc[:,1:]